In [41]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import pandas as pd
import re
from datetime import datetime
import sys 

quintile_mapping = {1: 'F2', 2: 'F1', 3: 'C', 4: 'B1', 5: 'B2'}

from Convert_to_Profiles_Functions import *
from Get_Events_Functions import *

gauge_nums = range(0,1294)
em ='bb216'
time_period='Future'

In [40]:
for em in ['bb216']:
    print(em)
    base_fp = f"/nfs/a161/gy17m2a/PhD/ProcessedData/"
    if em == 'nimrod':
        profiles_fp = f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/NIMROD_30mins/WholeYear/"
    else:
        profiles_fp = f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/UKCP18_30mins/{time_period}/{em}/"

    with open(profiles_fp + "df.pkl", 'rb') as file:
        df =pickle.load(file)

    # Create version without nulls    
    df_withoutnulls = df[df['precip'].notnull()].copy()       

    df_withoutnulls[['dimensionless_cumulative_rainfall', 'dimensionless_cumulative_times']] = df_withoutnulls['precip'].apply(create_huff_curves).apply(pd.Series)
    df_withoutnulls[['interpolated_cumulative_rainfall', 'interpolated_times']] = df_withoutnulls['dimensionless_cumulative_rainfall'].apply(lambda x: interpolate_rainfall(x,12)).apply(pd.Series)
    df_withoutnulls[['irain_14vals', 'iran_times_14vals']] = df_withoutnulls['interpolated_cumulative_rainfall'].apply(lambda x: create_irain_profile(x, True)).apply(pd.Series)
    df_withoutnulls[['irain', 'iran_times']] = df_withoutnulls['interpolated_cumulative_rainfall'].apply(lambda x: create_irain_profile(x, False)).apply(pd.Series)

    # # Add quintile cats
    df_withoutnulls['max_quintile_molly'] = df_withoutnulls['precip'].apply(find_max_quintile)
    df_withoutnulls['max_quintile_steef'] = df_withoutnulls['precip'].apply(analyse_event)
    df_withoutnulls['max_quintile_raw_rain'] = df_withoutnulls['precip'].apply(lambda x: find_part_with_most_rain(x, 5))
    df_withoutnulls[['max_quintile_steef', 'irain_profile_12_Steef']] = df_withoutnulls['precip'].apply(lambda x: pd.Series(analyse_event(x)))

    # Add loading cats
    df_withoutnulls['Loading_profile_raw_rain'] = df_withoutnulls['max_quintile_raw_rain'].map(quintile_mapping)
    df_withoutnulls['Loading_profile_molly'] =df_withoutnulls['max_quintile_molly'].map(quintile_mapping)
    df_withoutnulls['Loading_profile_steef'] =df_withoutnulls['max_quintile_steef'].map(quintile_mapping)

    # Save
    with open(profiles_fp + "df_nonulls.pkl", 'wb') as file:
        pickle.dump(df_withoutnulls, file)    

    with open(profiles_fp + "df.pkl", 'wb') as file:
        pickle.dump(df, file)  

bb216
